# Lab 9-1: MNIST Softmax&nn
이 파일은 모두딥 시즌2의 9-1,2 내용에 해당합니다

## Imports

In [ ]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

device = 'cuda' if torch.cuda.is_available() else 'cpu'

#for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

## Data

In [ ]:
#parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 10000

In [ ]:
#MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform= transforms.ToTensor(),
                          download= True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform= transforms.ToTensor(),
                          download= True)

#datalodaer
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

## Softmax

### Model (Softmax)

In [ ]:
linear = torch.nn.Linear(28*28, 10, bias = True).to(device)

#initialization
torch.nn.init.normal_(linear.weight)

#cost & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(linear.parameters(), lr= learning_rate)

### Training & Testing (Softmax)

In [ ]:
total_batch = len(data_loader)

for epoch in range(training_epochs):
  avg_cost = 0

  for X, Y in data_loader:
    # reshape input image into [batch_size by 784]
    # label is not one-hot encoded
    X = X.view(-1, 28*28).to(device)
    Y = Y.to(device)

    hypothesis = linear(X)
    cost = criterion(hypothesis, Y)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch
  
  print('Epoch: {:4d} Cost: {:.9f}'.format(epoch+1, avg_cost))
print('Learning Finished.')

Epoch:    1 Cost: 16.252239227
Epoch:    2 Cost: 15.525712967
Epoch:    3 Cost: 14.824342728
Epoch:    4 Cost: 14.151560783
Epoch:    5 Cost: 13.512964249
Epoch:    6 Cost: 12.906812668
Epoch:    7 Cost: 12.335800171
Epoch:    8 Cost: 11.800431252
Epoch:    9 Cost: 11.299505234
Epoch:   10 Cost: 10.832867622
Epoch:   11 Cost: 10.398482323
Epoch:   12 Cost: 9.993090630
Epoch:   13 Cost: 9.613684654
Epoch:   14 Cost: 9.257011414
Epoch:   15 Cost: 8.920917511
Learning Finished.


In [ ]:
#test
with torch.no_grad():
  X_test = mnist_test.data.view(-1, 28*28).float().to(device)
  Y_test = mnist_test.targets.to(device)

  prediction = linear(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy:', accuracy.item())

  #Get one and predict
  r = random.randint(0, len(mnist_test) - 1)
  X_single = mnist_test.data[r:r+1].view(-1, 28*28).float().to(device)
  Y_single = mnist_test.targets[r:r+1].to(device)

  print('Label:', Y_single.item())
  single_prediction = linear(X_single)
  print('Prediction:', torch.argmax(single_prediction, 1).item())

Accuracy: 0.12453333288431168
Label: 9
Prediction: 4


## nn

### Model (nn)

In [ ]:
#nn layers
linear1 = torch.nn.Linear(28*28, 256, bias= True)
linear2 = torch.nn.Linear(256, 256, bias= True)
linear3 = torch.nn.Linear(256, 10, bias= True)

relu = torch.nn.ReLU()

#initialization
torch.nn.init.normal_(linear1.weight)
torch.nn.init.normal_(linear2.weight)
torch.nn.init.normal_(linear3.weight)

#model
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3).to(device) #linear3이후에는 softmax가 적용되므로 relu가 x

#cost & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)

### Training & Testing (nn)

In [ ]:
total_batch = len(data_loader)

for epoch in range(training_epochs):
  avg_cost = 0

  for X, Y in data_loader:
    # reshape input image into [batch_size by 784]
    # label is not one-hot encoded
    X = X.view(-1, 28*28).to(device)
    Y = Y.to(device)

    hypothesis = model(X)
    cost = criterion(hypothesis, Y)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch
  
  print('Epoch: {:4d} Cost: {:.9f}'.format(epoch+1, avg_cost))
print('Learning Finished.')

Epoch:    1 Cost: 1137.021728516
Epoch:    2 Cost: 763.188171387
Epoch:    3 Cost: 568.009277344
Epoch:    4 Cost: 440.408630371
Epoch:    5 Cost: 346.120727539
Epoch:    6 Cost: 281.912597656
Epoch:    7 Cost: 239.317779541
Epoch:    8 Cost: 208.056930542
Epoch:    9 Cost: 183.941009521
Epoch:   10 Cost: 165.368011475
Epoch:   11 Cost: 150.524017334
Epoch:   12 Cost: 138.510574341
Epoch:   13 Cost: 128.500823975
Epoch:   14 Cost: 120.081802368
Epoch:   15 Cost: 112.788536072
Learning Finished.


In [ ]:
#test
with torch.no_grad():
  X_test = mnist_test.data.view(-1, 28*28).float().to(device)
  Y_test = mnist_test.targets.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy:', accuracy.item())

  #Get one and predict
  r = random.randint(0, len(mnist_test) - 1)
  X_single = mnist_test.data[r:r+1].view(-1, 28*28).float().to(device)
  Y_single = mnist_test.targets[r:r+1].to(device)

  print('Label:', Y_single.item())
  single_prediction = linear(X_single)
  print('Prediction:', torch.argmax(single_prediction, 1).item())

Accuracy: 0.7528499960899353
Label: 4
Prediction: 3
